In [0]:
from pyspark.sql import SparkSession

In [0]:
sc=SparkSession.builder.appName("claasification").getOrCreate()

In [0]:
data=sc.read.csv('/FileStore/tables/diabetes.csv',inferSchema=True,header=True)

In [0]:
data.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
col=data.columns
print(col)

['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']


In [0]:
ve=VectorAssembler(inputCols=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'],outputCol='feature')

In [0]:
newdata=ve.transform(data)

In [0]:
newdata.show(5)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|             feature|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|[6.0,148.0,72.0,3...|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|[1.0,85.0,66.0,29...|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|[8.0,183.0,64.0,0...|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|[1.0,89.0,66.0,23...|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|[0.0,137.0,40.0,3...|
+-----------+-------+-----------

In [0]:
train_data,test_data=newdata.randomSplit([0.8,0.2])

In [0]:
newdata.count()

Out[14]: 768

In [0]:
train_data.count()

Out[16]: 624

In [0]:
test_data.count()

Out[17]: 144

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

In [0]:
algo=DecisionTreeClassifier(featuresCol='feature',labelCol='Outcome')

In [0]:
model=algo.fit(train_data)

In [0]:
test_pred=model.transform(test_data)

In [0]:
test_pred.show(10)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+-------------+--------------------+----------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|             feature|rawPrediction|         probability|prediction|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+--------------------+-------------+--------------------+----------+
|          0|     73|            0|            0|      0|21.1|                   0.342| 25|      0|(8,[1,5,6,7],[73....|  [129.0,2.0]|[0.98473282442748...|       0.0|
|          0|     84|           64|           22|     66|35.8|                   0.545| 21|      0|[0.0,84.0,64.0,22...|  [81.0,20.0]|[0.80198019801980...|       0.0|
|          0|     84|           82|           31|    125|38.2|                   0.233| 23|      0|[0.0,84.0,82.0,31...|  [81.0,20.0]|[0.80198019801980...|       0.0

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-1370901595691214> in <module>
----> 1 print(BinaryClassificationEvaluator(test_pred))

/databricks/spark/python/pyspark/__init__.py in wrapper(self, *args, **kwargs)
    110     def wrapper(self, *args, **kwargs):
    111         if len(args) > 0:
--> 112             raise TypeError("Method %s forces keyword arguments." % func.__name__)
    113         self._input_kwargs = kwargs
    114         return func(self, **kwargs)

TypeError: Method __init__ forces keyword arguments.